In [ ]:
import pandas as pd
from pandasql import sqldf
from tqdm.notebook import tqdm
import numpy as np
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
import matplotlib

font = {'family' : 'normal',
        'size'   : 11}

matplotlib.rc('font', **font)


## Deriving the ADHD data from medsy01

df = pd.read_csv(r"X:\CompSci\ResearchProjects\AJoshi\aj2151 - AJoshi\Matt Bearham\Package_1202749\medsy01.txt",sep='\t')

mysql = lambda q: sqldf(q, globals())

adhd_terms = [
    'dexedrine',
    'zenzedi',
    'adderall',
    'focalin',
    'methylin',
    'ritalin',
    'methylphenidate',
    'dyanavel',
    ' amphetamine',
    'dextroamphetamine',
    'evekeo',
    'mydayis',
    'vyvanse',
    'aptensio',
    'concerta',
    'lisdexamfetamine',
    'quillivant',
    'quillichew',
    'azstarys',
    'metadate',
    'cotempla',
    'atomoxetine',
    'strattera',
    'guanfacine',
    'tenex',
    'intuniv',
    'clonidine',
    'kapvay']

data = []
for i in tqdm(range(15),desc='fields'):
    for term in tqdm(adhd_terms,desc='terms',leave=False):
        temp = df.copy()
        temp['match'] = temp[f'med{i+1}_rxnorm_p'].str.lower().str.contains(term)
        result = temp[temp['match'] == True]['subjectkey']
        result = pd.DataFrame(result)
        result['medication'] = term
        #result = mysql(f"SELECT DISTINCT subjectkey FROM df WHERE med{i+1}_rxnorm_p LIKE '%{term}%' COLLATE utf8_general_ci")
        data.append(result)

adhd_subjects = pd.concat(data).reset_index().drop(['index'],axis=1)

pivot = pd.pivot_table(adhd_subjects,index='subjectkey',columns='medication',aggfunc=len,fill_value=0)

medications = pd.DataFrame()
medications['methylphenidate'] = pivot['aptensio'] + pivot['concerta'] + pivot['cotempla'] + pivot['focalin'] + pivot['metadate'] + pivot['methylin'] + pivot['methylphenidate'] + pivot['quillichew'] + pivot['quillivant'] + pivot['ritalin']
medications['dexamphetamine'] = pivot['adderall'] + pivot[' amphetamine'] + pivot['dexedrine'] + pivot['dyanavel'] + pivot['evekeo'] + pivot['zenzedi']
medications['lisdexamphetamine'] = pivot['lisdexamfetamine'] + pivot['vyvanse']
medications['atomoxetine'] = pivot['atomoxetine'] + pivot['strattera']
medications['guanfacine'] = pivot['guanfacine'] + pivot['tenex'] + pivot['intuniv']
medications['clonidine'] = pivot['kapvay'] + pivot['clonidine']
medications = medications.clip(0,1)

adhd_subjects_distinct = pd.concat(data)['subjectkey'].drop_duplicates().reset_index()

num_adhd = len(adhd_subjects_distinct)
num_not = len(df['subjectkey'].unique())-len(adhd_subjects_distinct)
print(f'Number with ADHD: {num_adhd}')
print(f'Number without ADHD: {num_not}')

plt.figure(figsize=(8.5,5.5))
plt.subplot(1,2,1)
plt.pie([num_not,num_adhd],labels=['Participants without ADHD','Participants with ADHD'],
        autopct='%1.1f%%',labeldistance=None,explode=(0,0.2))
plt.title("a) Prevalence of ADHD",loc="left")
plt.legend(loc=3)
plt.subplot(1,2,2)
plt.bar(medications.columns,medications.sum())
plt.title("b) ADHD medication counts",loc="left")
plt.xticks(rotation=45)

plt.savefig('Figures/rate_adhd')
plt.show()

print(f"{len(medications[medications.sum(axis=1)==2])} participants on 2 different ADHD medications")
print(f"{len(medications[medications.sum(axis=1)==3])} participants on 3 different ADHD medications")
print(f"{len(medications[medications.sum(axis=1)==4])} participants on 4 different ADHD medications")
print(f"{len(medications[medications.sum(axis=1)==5])} participants on 5 different ADHD medications")
print(f"{len(medications[medications.sum(axis=1)>5])} participants on more than 5 different ADHD medications")

doubles = medications[medications.sum(axis=1)==2]
triples = medications[medications.sum(axis=1)==3]
quartets = medications[medications.sum(axis=1)==4]

count_doubles = doubles.replace(1,'o').groupby(doubles.columns.tolist(),as_index=False).size()\
    .reset_index().rename(columns={0:'records'}).sort_values(by='records',ascending=False)\
    .replace(0,'-')
count_triples = triples.replace(1,'o').groupby(triples.columns.tolist(),as_index=False).size()\
    .reset_index().rename(columns={0:'records'}).sort_values(by='records',ascending=False)\
    .replace(0,'-')
count_quartets = quartets.replace(1,'o').groupby(quartets.columns.tolist(),as_index=False).size()\
    .reset_index().rename(columns={0:'records'}).sort_values(by='records',ascending=False)\
    .replace(0,'-')

crossfreq = medications.T.dot(medications)
crossfreq.to_csv('crossfreq.csv')

def generate_nice_table(dataframe):
    nice_table = []
    for index,row in dataframe.iterrows():
        drugs = []
        for column in row.index:
            if row[column] == 'o':
                drugs.append(column)
        string =  ', '.join(drugs)
        nice_table.append([string,row['records']])
    return pd.DataFrame(nice_table).rename(columns={0:'drugs',1:'counts'})
    
print(generate_nice_table(count_doubles))
print(generate_nice_table(count_triples))
print(generate_nice_table(count_quartets))

## eatqp01

temprament = pd.read_csv(r"X:\CompSci\ResearchProjects\AJoshi\aj2151 - AJoshi\Matt Bearham\Package_1202749\abcd_eatqp01.txt",sep='\t')
temprament['adhd'] = [1 if x in adhd_subjects_distinct['subjectkey'].tolist() else 0 for x in temprament['subjectkey']]

for i in ['eatq_finish_p','eatq_turn_taking_p','eatq_open_present_p','eatq_ski_slope_p','eatq_before_hw_p','eatq_distracted_p','eatq_impulse_p','eatq_social_p','eatq_hardly_sad_p','eatq_try_focus_p','eatq_no_criticize_p','eatq_puts_off_p','eatq_sidetracked_p','eatq_not_shy_p','eatq_meet_p','eatq_rides_scared_p']:
    temprament[i].replace({1:5,2:4,4:2,5:1},inplace=True)
    
temprament = temprament.dropna()
descriptions = temprament.iloc[0]
temprament = temprament.iloc[1:]

X = temprament.iloc[:,10:72]
y = temprament['adhd']
print(y.value_counts())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=150)

model = LogisticRegression(class_weight={0:1258,1:9073})
model.fit(X_train,y_train)
importance = model.coef_[0]

coefficients = []
for i in range(len(importance)):
    coefficients.append((X.columns[i],importance[i]))
    
to_show = 14

sorted_coef = sorted(coefficients,key=lambda tup:abs(tup[1]))
bottom_5 = sorted_coef[:to_show]
top_5 = sorted_coef[-to_show:]

print(f'*** The {to_show} least predictive measures were (odds ratio): ***')
for index,(label,value) in enumerate(bottom_5):
    print(f"{len(coefficients)-index}. {descriptions[label].split(' / ')[0]}: {round(np.exp(value),3)}")
print(f'\n*** The {to_show} most predictive measures were (odds ratio): ***')
for index,(label,value) in enumerate(top_5):
    print(f"{to_show-index}. {descriptions[label].split('. / ')[0]}: {round(np.exp(value),3)}")
    
## Optimising the weighting of the model

regular = np.arange(0,2,0.05)
zoomed = np.arange(0.05,0.25,0.001)

scores_weight = []

for i in tqdm(regular,desc="varying weights"):
    
    result = []
    
    model = LogisticRegression(class_weight={0:i,1:1},max_iter=1000)
    labels = [entry[0] for entry in sorted_coef]

    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    matrix = confusion_matrix(y_test, y_pred)

    result.append(i)
    result.append(accuracy_score(y_test,y_pred))
    result.append(roc_auc_score(y_test, y_pred))
    result.append(recall_score(y_test,y_pred))
    for i in range(2):
        for k in range(2):
            result.append(matrix[i][k])
    result.append(precision_score(y_test,y_pred))
    result.append(f1_score(y_test,y_pred))
    
    scores_weight.append(result)
    
scores_weight_df = pd.DataFrame(scores_weight)
scores_weight_df.columns = ['i','accuracy','auc','recall','tn','fp','fn','tp','precision','f1']

plt.figure(figsize=(8.5,15))
plt.subplot(3,1,1)
plt.title("a) AUC by weighting ratio (0 to 2, step = 0.02)")
plt.plot(regular,scores_weight_df['auc'])
plt.xlabel('Weighting Ratio (:1)')
plt.ylabel('Area Under Curve')

## Narrower set of weights around previous peak

scores_weight_zoom = []

for i in tqdm(zoomed,desc="zoomed weights"):
    
    result = []
    
    model = LogisticRegression(class_weight={0:i,1:1},max_iter=1000)
    labels = [entry[0] for entry in sorted_coef]

    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    matrix = confusion_matrix(y_test, y_pred)

    result.append(i)
    result.append(accuracy_score(y_test,y_pred))
    result.append(roc_auc_score(y_test, y_pred))
    result.append(recall_score(y_test,y_pred))
    for i in range(2):
        for k in range(2):
            result.append(matrix[i][k])
    result.append(precision_score(y_test,y_pred))
    result.append(f1_score(y_test,y_pred))
    scores_weight_zoom.append(result)

scores_weight_zoom_df = pd.DataFrame(scores_weight_zoom)
scores_weight_zoom_df.columns = ['i','accuracy','auc','recall','tn','fp','fn','tp','precision','f1']
print(scores_weight_zoom_df.iloc[scores_weight_zoom_df.idxmax()[2]])

plt.subplot(3,1,2)
plt.title("b) AUC by weighting ratio (0 to 0.5, step = 0.001)")
plt.plot(zoomed,scores_weight_zoom_df['auc'])
plt.xlabel('Weighting Ratio (:1)')
plt.ylabel('Area Under Curve')

## Varying the number of features used

scores = []

for j in tqdm(range(1,len(sorted_coef)+1),desc="varying features used"):

    result = []

    model = LogisticRegression(class_weight={0:1140,1:9275},max_iter=1000)
    labels = [entry[0] for entry in sorted_coef]
    subset = labels[-j:]

    model.fit(X_train[subset],y_train)

    y_pred = model.predict(X_test[subset])
    matrix = confusion_matrix(y_test, y_pred)

    result.append(j)
    result.append(accuracy_score(y_test,y_pred))
    result.append(roc_auc_score(y_test, y_pred))
    result.append(recall_score(y_test,y_pred))
    for i in range(2):
        for k in range(2):
            result.append(matrix[i][k])
    result.append(precision_score(y_test,y_pred))
    result.append(f1_score(y_test,y_pred))
    scores.append(result)

scores_df = pd.DataFrame(scores)
scores_df.columns = ['features','accuracy','auc','recall','tn','fp','fn','tp','precision','f1']

plt.subplot(3,1,3)
plt.title("c) AUC by number of features used")
plt.plot(scores_df['auc'])
plt.xlabel('Number of Features')
plt.ylabel('Area Under Curve')
plt.savefig("Figures/features")
plt.show()

def nicer_confusion_matrix(given_y_test,given_y_pred):
    matrix = confusion_matrix(given_y_test,given_y_pred)
    dictionary = {'actual control':matrix[0],'actual ADHD':matrix[1]}
    df = pd.DataFrame(dictionary)
    df = df.set_index([['predicted control','predicted ADHD']])
    return df

svm_scores = []
svm_labels = []
for i in tqdm(np.arange(0,2,0.1),desc = "SVM weights"):

    svm_model = svm.SVC(class_weight={0:i,1:1})
    svm_model.fit(X_train,y_train)
    svm_pred = svm_model.predict(X_test)
    svm_matrix = nicer_confusion_matrix(y_test, svm_pred)
    svm_acc = accuracy_score(y_test,svm_pred)
    svm_auc = roc_auc_score(y_test,svm_pred)
    svm_recall = recall_score(y_test,svm_pred)
    svm_precision = precision_score(y_test,y_pred)
    svm_f1 = f1_score(y_test,y_pred)
    svm_scores.append(svm_auc)
    svm_labels.append(i)

plt.figure(figsize = (8.5,15))
plt.subplot(3,1,1)
plt.plot(svm_labels,svm_scores)
plt.xlabel("Weighting ratio (:1)")
plt.ylabel("AUC of model")
plt.title("a) AUC of SVM model with varying class_weight")

svm_scores_2 = []
svm_labels_2 = []
for i in tqdm(np.arange(0.05,0.2,0.01),desc = "SVM weights"):

    svm_model = svm.SVC(class_weight={0:i,1:1})
    svm_model.fit(X_train,y_train)
    svm_pred = svm_model.predict(X_test)
    svm_matrix = nicer_confusion_matrix(y_test, svm_pred)
    svm_acc = accuracy_score(y_test,svm_pred)
    svm_auc = roc_auc_score(y_test,svm_pred)
    svm_recall = recall_score(y_test,svm_pred)
    svm_precision = precision_score(y_test,y_pred)
    svm_f1 = f1_score(y_test,y_pred)
    svm_scores_2.append(svm_auc)
    svm_labels_2.append(i)

plt.subplot(3,1,2)
plt.plot(svm_labels_2,svm_scores_2)
plt.xlabel("Weighting ratio (:1)")
plt.ylabel("AUC of model")
plt.title("b) AUC of SVM model with varying class_weight")

svm_scores_3 = []
svm_labels_3 = []
for i in tqdm(range(1,len(sorted_coef)+1),desc="varying features used"):

    svm_model = svm.SVC(class_weight={0:0.11,1:1})
    
    labels = [entry[0] for entry in sorted_coef]
    subset = labels[-i:]

    svm_model.fit(X_train[subset],y_train)
    
    svm_pred = svm_model.predict(X_test[subset])
    svm_matrix = nicer_confusion_matrix(y_test, svm_pred)
    svm_acc = accuracy_score(y_test,svm_pred)
    svm_auc = roc_auc_score(y_test,svm_pred)
    svm_recall = recall_score(y_test,svm_pred)
    svm_precision = precision_score(y_test,y_pred)
    svm_f1 = f1_score(y_test,y_pred)
    svm_scores_3.append(svm_auc)
    svm_labels_3.append(i)

print(svm_labels_3[svm_scores_3.index(max(svm_scores_3))])

plt.subplot(3,1,3)
plt.title("SVM AUC by number of features used")
plt.xlabel("Top n features used")
plt.ylabel("AUC Score")
plt.plot(svm_labels_3,svm_scores_3)
plt.savefig("Figures/svm_features_vary")
plt.show()

print(np.arange(0.05,0.2,0.01)[svm_scores_2.index(max(svm_scores_2))])

svm_model = svm.SVC(class_weight={0:0.11,1:1})

labels = [entry[0] for entry in sorted_coef]
subset = labels[-16:]
svm_model.fit(X_train[subset],y_train)

svm_pred = svm_model.predict(X_test[subset])
svm_matrix = nicer_confusion_matrix(y_test, svm_pred)
svm_acc = accuracy_score(y_test,svm_pred)
svm_auc = roc_auc_score(y_test,svm_pred)
svm_recall = recall_score(y_test,svm_pred)

svm_matrix = nicer_confusion_matrix(y_test, svm_pred)
svm_acc = accuracy_score(y_test,svm_pred)
svm_auc = roc_auc_score(y_test,svm_pred)
svm_recall = recall_score(y_test,svm_pred)
svm_precision = precision_score(y_test,y_pred)
svm_f1 = f1_score(y_test,y_pred)

print(svm_matrix)
print(f"Accuracy: {svm_acc}")
print(f"AUC: {svm_auc}")
print(f"Recall: {svm_recall}")
print(f"Precision: {svm_precision}")
print(f"F1: {svm_f1}")

forest_scores = []
forest_labels = []
for i in tqdm(np.arange(0,2,0.1),desc = "Forest weights"):
    forest = RandomForestClassifier(class_weight={0:i,1:1})
    forest.fit(X_train,y_train)
    forest_pred = forest.predict(X_test)
    forest_auc = roc_auc_score(y_test,forest_pred)
    forest_scores.append(forest_auc)
    forest_labels.append(i)
    
plt.figure(figsize=(12,4))
plt.plot(forest_labels,forest_scores)
plt.xlabel("Weight ratio (:1)")
plt.ylabel("AUC")
plt.title("Decision Forest AUC changing with weighting ratio")
plt.savefig("Figures/forest_weights")
plt.show()

forest = RandomForestClassifier(class_weight={0:0.11,1:1})
forest.fit(X_train,y_train)
forest_pred = forest.predict(X_test)
forest_matrix = nicer_confusion_matrix(y_test, forest_pred)
forest_acc = accuracy_score(y_test,forest_pred)
forest_auc = roc_auc_score(y_test,forest_pred)
forest_recall = recall_score(y_test,forest_pred)
forest_precision = f1_score(y_test,forest_pred)
forest_f1 = precision_score(y_test,forest_pred)

print(forest_matrix)
print(f"Accuracy: {forest_acc}")
print(f"AUC: {forest_auc}")
print(f"Recall: {forest_recall}")
print(f"Precision: {forest_precision}")
print(f"F1 score: {forest_f1}")
